In [40]:
# type: ignore
import transformers
import math
import statistics
from tqdm import tqdm
import torch.nn.functional as F
import torch
import os
import numpy as np
import pandas as pd
import seaborn as sns
from transformers import pipeline
from torch.utils.data.dataloader import default_collate
import torchvision
from torchvision.models import resnet101
from torchvision.models import resnet50
from torch.utils.data import DataLoader
import av
from torch import nn, optim
from torchvision import transforms
from torchvision.datasets import UCF101
import time
import re
import configparser


In [41]:
start_time = time.time()

In [42]:
def read_config(config_file):
    config = configparser.ConfigParser()
    config.read(config_file)
    return config


In [54]:
config = read_config('config.ini')
runPipe = ''
if config.sections() == []:
    config = read_config('../config.ini')
    runPipe = '.'

epochs = config.getint('training', 'epochs')
train_subset = config.get('data', 'train_subset_100k')
test_subset = config.get('data', 'test_subset_20k')
train_subset_rest = config.get('data', 'train_subset_100k_rest')
test_subset_rest = config.get('data', 'test_subset_20k_rest')
batch_size = config.getint('training', 'batch_size')
train_size = config.getint('training', 'train_size')

In [48]:
def custom_collate(batch):
    filtered_batch = []
    for video, _, label in batch:
        filtered_batch.append((video, label))
    return torch.utils.data.dataloader.default_collate(filtered_batch)

def divide_by_255(x):
            return x / 255.

def permute_channels(x):
    return x.permute(0, 3, 1, 2)

def interpolate(x):
    return nn.functional.interpolate(x, (240, 320))

tfs = transforms.Compose([
    transforms.Lambda(divide_by_255),
    transforms.Lambda(permute_channels),
    transforms.Lambda(interpolate),
])

In [56]:
# data_files is an array of data paths and is passed from executable setup.py

# Load train_dataset
with open(runPipe + './data/train_dataset.pt', 'rb') as f:
    train_dataset = torch.load(f)

# Load test_dataset
with open(runPipe + './data/test_dataset.pt', 'rb') as f:
    test_dataset = torch.load(f)

# Load train_dataset_100k
with open(runPipe + train_subset, 'rb') as f:
    train_set = torch.load(f)

# Load test_dataset_20k
with open(runPipe + test_subset, 'rb') as f:
    test_set = torch.load(f)

# Load train_dataset_100k_rest
with open(runPipe + train_subset_rest, 'rb') as f:
    test_set_org = torch.load(f)

# Load test_dataset_20k_rest
with open(runPipe + test_subset_rest, 'rb') as f:
    test_set2 = torch.load(f)
    
train_loader_sub = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True,
                                           collate_fn=custom_collate)
test_loader_sub = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True,
                                          collate_fn=custom_collate)
# train_loader_full = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
#                                            collate_fn=custom_collate)
# test_loader_full = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True,
#                                           collate_fn=custom_collate)

In [ ]:
print(f"Total number of train samples: {len(train_dataset)}")
print(f"Total number of test samples: {len(test_dataset)}")
print(f"Total number of (train) batches: {len(train_loader_sub)}")
print(f"Total number of (test) batches: {len(test_loader_sub)}")

In [ ]:
class HLSTMCNN(nn.Module):
    def __init__(self, num_classes=101):
        super(HLSTMCNN, self).__init__()
        self.resnet = resnet50(pretrained=True)
        self.resnet.fc = nn.Sequential(nn.Linear(self.resnet.fc.in_features, 300))
        self.lstm = nn.LSTM(input_size=300, hidden_size=256, num_layers=3)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x_3d):
        hidden = None

        # Iterate over each frame of a video in a video of batch * frames * channels * height * width
        for t in range(x_3d.size(1)):
            with torch.no_grad():
                x = self.resnet(x_3d[:, t])
            # Pass latent representation of frame through lstm and update hidden state
            out, hidden = self.lstm(x.unsqueeze(0), hidden)

        # Get the last hidden state (hidden is a tuple with both hidden and cell state in it)
        x = self.fc1(hidden[0][-1])
        x = F.relu(x)
        x = self.fc2(x)

        return x

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")  # use gpu if available
target_model = HLSTMCNN().to(device=device)
optimiser=torch.optim.SGD(target_model.parameters(),lr=0.01,momentum=0.9)
cost = torch.nn.CrossEntropyLoss()

In [ ]:
def target_train(train_loader, target_model, optimiser):
    target_model.train()
    size = len(train_loader.dataset)
    correct = 0
    total_loss=[]
    for batch, (X, Y) in enumerate(tqdm(train_loader)):
        X, Y = X.to(device=device), Y.to(device=device)
        #print(X.shape, Y.shape)
        optimiser.zero_grad()
        pred = target_model(X)
        #print(pred, Y)
        #pred = pred.flatten()
        loss = cost(pred, Y)
        loss.backward()
        optimiser.step()
        _, output = torch.max(pred, 1)
        #print(output, Y)
        correct+= (output == Y).sum().item()
        total_loss.append(loss.item())
        #batch_count+=batch
        #correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    #print(correct, size)
    correct /= size
    loss= sum(total_loss)/(batch+1)
    result_train = correct
    
    # train samples is a passed in parameter from executable setup.py
    torch.save(target_model, runPipe + f'./models/base_model_{train_size}k.pt')

    print(f'\nTraining Performance:\nacc: {(100*correct):>0.1f}%, avg loss: {loss:>8f}\n')

    return loss, result_train


In [ ]:
def target_utility_top5(test_loader, target_model, batch_size = batch_size):
   size = len(test_loader.dataset)
   target_model.eval()
   test_loss, correct = 0, 0
   correct = 0
   topk_correct=0
   total=0
   counter_a=0
   #with torch.no_grad():
   for batch, (X, Y) in enumerate(tqdm(test_loader)):
       X, Y = X.to(device=device), Y.to(device=device)
       X.requires_grad = True
       pred = target_model(X)
       #print("Y is: ", Y)
       counter_a=counter_a+1
       #test_loss += cost(pred, Y).item()
       #correct += (pred.argmax(1)==Y).type(torch.float).sum().item()
       #data, target = data.to(device), target.to(device)
       # Set requires_grad attribute of tensor. Important for Attack
       total += Y.size(0)
       # Forward pass the data through the model
       _, output_res = torch.max(pred, -1)
       #print("pred is: ",output_res)
       correct += ((output_res) == Y).sum().item()
       top_k=5
       for test_sample in range(X.shape[0]):
           val, idx=torch.topk(pred[test_sample, :], top_k)
           for check in range(top_k):
               if (idx[check]==Y[test_sample]):
                   topk_correct+=1


   # Calculate final accuracy for this epsilon
   topk_acc= topk_correct/float(total)
   final_acc = correct/float(total)
   print(f"Target Model Top-1 Acc = {correct} / {total} = {final_acc}, Top-5 Acc= {topk_correct} / {total}={topk_acc}")


   # Return the accuracy and an adversarial example
   return final_acc, topk_acc



In [ ]:
loss_train_tr, loss_test_tr=[],[]
for t in tqdm(range(target_epochs)):
    print(f'Epoch {t+1}\n-------------------------------')
    print("+++++++++Target Training Starting+++++++++")
    tr_loss, result_train=target_train(train_loader_sub, target_model, optimiser)
    loss_train_tr.append(tr_loss)

    # final_acc, topk_acc=target_utility_top5(test_loader_sub, target_model, batch_size = batch_size)

    print(
    f'CNN Training update after {t+1} epochs',
    f'''
        Train loss: {tr_loss}
        Train accuracy: {round(result_train/100, 2)}
     '''
        # Top-1 Test accuracy: {final_acc}
        # Top-5 Test accuracy: {topk_acc}
    )


In [ ]:
hours =  (time.time()-start_time)//3600
minutes = ((time.time()-start_time)%3600)//60
seconds = ((time.time()-start_time)%3600)%60
date = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [ ]:
final_acc_top1, final_acc_top5=target_utility_top5(test_loader_sub, target_model, batch_size = batch_size)
# save to a file

with open('../results/results.txt', 'w') as f:
    f.write(f"\n----------------------------------------------\n")
    f.write(f"Training Finished at: {date}\n")
    f.write(f"Base Model Training Samples: {train_size}\n")
    f.write(f"Final Train Loss: {round(loss_train_tr[-1], 2)}\n")
    f.write(f"Final Test Loss: {round(loss_test_tr[-1], 2)}\n")
    f.write(f"Final Top-1 Accuracy: {final_acc_top1}\n")
    f.write(f"Final Top-5 Accuracy: {final_acc_top5}\n")
    f.write(f"Total Training Time: {hours} Hours, {minutes} Minutes, {seconds} Seconds\n")
